In [1]:
import tensorflow as tf
tf.test.is_gpu_available(
    cuda_only=False, min_cuda_compute_capability=None
)

S:\Anaconda\envs\tf-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
S:\Anaconda\envs\tf-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
S:\Anaconda\envs\tf-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
S:\Anaconda\envs\tf-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (

True

In [2]:
from tensorflow.keras.datasets import cifar10

from tensorflow.keras.models import Model

from tensorflow.keras.layers import Input, Convolution2D, MaxPooling2D, Dense, Dropout, Flatten

from tensorflow.python.keras import utils

import numpy as np

from datetime import datetime

In [3]:
batch_size = 32 # in each iteration, we consider 32 training examples at once

num_epochs = 200 # we iterate 200 times over the entire training set

kernel_size = 5 # we will use 3x3 kernels throughout

pool_size = 2 # we will use 2x2 pooling throughout

conv_depth_1 = 32 # we will initially have 32 kernels per conv. layer...

conv_depth_2 = 64 # ...switching to 64 after the first pooling layer

drop_prob_1 = 0.25 # dropout after pooling with probability 0.25

drop_prob_2 = 0.5 # dropout in the dense layer with probability 0.5

hidden_size = 512 # the dense layer will have 512 neurons

In [4]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data() # fetch CIFAR-10 data


num_train, depth, height, width = X_train.shape # there are 50000 training examples in CIFAR-10

num_test = X_test.shape[0] # there are 10000 test examples in CIFAR-10

num_classes = np.unique(y_train).shape[0] # there are 10 image classes


X_train = X_train.astype('float32')

X_test = X_test.astype('float32')

X_train /= np.max(X_train) # Normalise data to [0, 1] range

X_test /= np.max(X_train) # Normalise data to [0, 1] range


Y_train = utils.to_categorical(y_train, num_classes) # One-hot encode the labels

Y_test = utils.to_categorical(y_test, num_classes) # One-hot encode the labels

In [5]:
import tensorflow.keras.backend as K
# to set the image order
K.set_image_data_format('channels_first')

In [6]:
inp = Input(shape=(depth, height, width)) # N.B. depth goes first in Keras


# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)

conv_1 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='relu')(inp)

conv_2 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='relu')(conv_1)

pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size), padding='same')(conv_2)

drop_1 = Dropout(drop_prob_1)(pool_1)


# Conv [64] -> Conv [64] -> Pool (with dropout on the pooling layer)

conv_3 = Convolution2D(conv_depth_2, kernel_size, kernel_size, padding='same', activation='relu')(drop_1)

conv_4 = Convolution2D(conv_depth_2, kernel_size, kernel_size, padding='same', activation='relu')(conv_3)

pool_2 = MaxPooling2D(pool_size=(pool_size, pool_size), padding='same')(conv_4)

drop_2 = Dropout(drop_prob_1)(pool_2)


# Now flatten to 1D, apply Dense -> ReLU (with dropout) -> softmax

flat = Flatten()(drop_2)

hidden = Dense(hidden_size, activation='relu')(flat)

drop_3 = Dropout(drop_prob_2)(hidden)

out = Dense(num_classes, activation='softmax')(drop_3)


model = Model(inp, out) # To define a model, just specify its input and output layers


model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function

              optimizer='nadam', # using the Adam optimiser

              metrics=['accuracy']) # reporting the accuracy

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [7]:
history = model.fit(X_train, Y_train, 
                    batch_size=batch_size, 
                    epochs=num_epochs, 
                    verbose=1, 
                    validation_split=0.1)

model.evaluate(X_test, Y_test, verbose=1) # Evaluate the trained model on the test set!

Train on 45000 samples, validate on 5000 samples
Epoch 1/200
45000/45000 [==============================] - 9s 209us/sample - loss: 2.0701 - acc: 0.1869 - val_loss: 1.9937 - val_acc: 0.2186
Epoch 2/200
45000/45000 [==============================] - 8s 175us/sample - loss: 1.9048 - acc: 0.2677 - val_loss: 1.8413 - val_acc: 0.2980
Epoch 3/200
45000/45000 [==============================] - 8s 177us/sample - loss: 1.8267 - acc: 0.3140 - val_loss: 1.7918 - val_acc: 0.3384
Epoch 4/200
45000/45000 [==============================] - 8s 176us/sample - loss: 1.7758 - acc: 0.3433 - val_loss: 1.7202 - val_acc: 0.3624
Epoch 5/200
45000/45000 [==============================] - 8s 176us/sample - loss: 1.7375 - acc: 0.3616 - val_loss: 1.6492 - val_acc: 0.3988
Epoch 6/200
45000/45000 [==============================] - 8s 175us/sample - loss: 1.7044 - acc: 0.3779 - val_loss: 1.6446 - val_acc: 0.3918
Epoch 7/200
45000/45000 [==============================] - 8s 175us/sample - loss: 1.6828 - acc: 0.3901 -

[436.0161907714844, 0.349]